In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [72]:
df2=pd.read_csv("df2.csv")

In [123]:
df3=df2[["subaward_action_date","top_subawardee_name","subaward_amount"]]
df3.subaward_action_date=df3.subaward_action_date.str[0:4]
df4= df3.groupby(by=['subaward_action_date',"top_subawardee_name"],as_index=False).\
        agg({'subaward_amount':"sum"}).\
    sort_values(by='subaward_action_date',ascending=False).\
    reset_index().\
    drop("index",axis=1)
df4.to_csv('df4.csv',index=False)

/Users/bruce/opt/miniconda3/envs/spyder-env-y/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [124]:
source = df2
brush = alt.selection_interval(encodings=["x"])
click = alt.selection_multi(fields=["top_subawardee_name"])

scale=alt.Scale(domain=["Alliant Techsystems Operations","Orbital Sciences", 
    "Ensign-Bickford Aerospace & Defense Company",
    "Pacific Scientific Energetic Materials Company (California)", 
    "Aerojet Rocketdyne","Ducommun Labarge Technologies",
    "Tri Models","Others"],range=["#e60049", "#0bb4ff", "#50e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#999999"])

color=alt.Color("top_subawardee_name:N",title="Main Subcontractors", scale=scale) 

scatter= alt.Chart(source).mark_point().encode(
    x=alt.X('subaward_action_date:T',title="Date", axis=alt.Axis(labelAngle=320,grid=False)),
    y=alt.Y('subaward_amount:Q',title="Subcontract Amounts",axis=alt.Axis(grid=False, titlePadding=20)),
    color=alt.condition(brush,color,alt.value('lightgray')),
).properties(
    width=650,
    height=300
).add_selection(brush).transform_filter(click)

bar = alt.Chart(source).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('sum(subaward_amount):Q',title="Subcontract Amounts",axis=alt.Axis(grid=False)),
    y=alt.Y("top_subawardee_name:N",title="Main Subcontractors", axis=alt.Axis(grid=False, titlePadding=220)),
    color=alt.condition(click,color,alt.value("lightgray"))
).properties(
    width=650,
    height=300
    # title="Sentiment results of Top20 stocks mentioned in Reddit (r/stocks)"
).add_selection(click).transform_filter(brush)

alt.vconcat(scatter, bar). \
    configure(font="Georgia").\
    configure_legend(titleColor='black', labelFontSize=17, titleFontSize=17,labelLimit=1000). \
    configure_axis(labelFontSize=15,titleFontSize=17,labelLimit=1000).\
    save('time2.html')


